In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import pymysql

conn = pymysql.connect(host='localhost', user='root', password='ehfkwl1!', db='newsdb', charset='utf8')
cursor = conn.cursor(pymysql.cursors.DictCursor)

In [3]:

sql = "SELECT * FROM `tb_news`;" #테이블명 적기
cursor.execute(sql)
result = cursor.fetchall()

#print(result)하면 무한 돌아감


In [4]:
db = pd.DataFrame(result)

In [15]:
#데이터 가져오기
db.head()

,ID,DomainID,MainCategory,SubCategory,WritedAt,Title,Content,URL,PhotoURL,Writer,Stickers,Press
0,1,1,스포츠,해외야구,2022-11-30 02:52:00,휴스턴 레전드의 일침 “야구는 컴퓨터가 아니라 사람이 하는 것”,휴스턴 애스트로스 레전드이자 미국 야구 명예의 전당 멤버인 제프 배그웰54은 분명한...,https://sports.news.naver.com/news?oid=410&aid...,['https://imgnews.pstatic.net/image/410/2022/1...,김재호,"{'좋아요': 5, '슬퍼요': 1, '화나요': 15, '팬이에요': '0', '...",MK스포츠
1,2,1,스포츠,골프,2022-11-30 00:05:00,[GD레슨] 진정한 실력자가 되기 위해 필요한 세 가지 기술 ②,내가 어렸을 때 우리 집 뒷마당의 나무에는 낡은 타이어가 고정되어 있어서 가끔씩 야...,https://sports.news.naver.com/news?oid=435&aid...,['https://imgnews.pstatic.net/image/435/2022/1...,김성준,"{'좋아요': 1, '슬퍼요': '0', '화나요': '0', '팬이에요': '0'...",골프다이제스트
2,3,1,스포츠,배구,2022-11-30 00:00:00,‘1무 1패’ 한국과 동병상련…카메룬 특급도 월드컵 시청 “지지 않아서 좋았다” [...,GS칼텍스 모마 KOVO 제공인천 기자 GS칼텍스 외국인선수 모마GS칼텍스도 지...,https://sports.news.naver.com/news?oid=109&aid...,['https://imgnews.pstatic.net/image/109/2022/1...,이후광,"{'좋아요': 12, '슬퍼요': 1, '화나요': 2, '팬이에요': 2, '후속...",OSEN
3,4,1,스포츠,일반,2022-11-30 00:02:00,[삼성화재배 AI와 함께하는 바둑 해설] 흑의 의도를 비틀다,〈본선 32강전〉 ○ 박정환 9단 ● 커제 9단 장면 2 장면 ②바둑이 자꾸만 추상...,https://sports.news.naver.com/news?oid=025&aid...,['https://imgnews.pstatic.net/image/025/2022/1...,,"{'좋아요': '0', '슬퍼요': '0', '화나요': '0', '팬이에요': '...",중앙일보
4,5,1,스포츠,해외야구,2022-11-29 03:00:00,"어제의 적이 동지? 벌랜더, 다저스 만난다",어제의 적이 내일의 동료가 될 수 있을까?뉴욕포스트 메이저리그 전문 칼럼니스트 존 ...,https://sports.news.naver.com/news?oid=410&aid...,['https://imgnews.pstatic.net/image/410/2022/1...,김재호,"{'좋아요': 14, '슬퍼요': '0', '화나요': 6, '팬이에요': 12, ...",MK스포츠


In [5]:
db.shape

(682288, 12)

In [6]:
data = db

In [8]:
#doc2vec model 학습하고 저장하는 함수
def make_doc2vec_models(tagged_data, tok, vector_size=128,window = 3, epochs=10, min_count=0,workers=4):
    model = Doc2Vec(tagged_data, vector_size=vector_size, window=window, epochs=epochs,min_count=min_count,workers=workers)
    model.save(f'./{tok}_news_model.doc2vec')

In [ ]:
# #데이터 전처리
# def get_preprocessing_data(data):
#     data.drop(['date','company','url'],axis = 1,inplace=True)

#     category_mapping = {
#         '경제' : 0,
#         '정치' : 1,
#         'IT과학' : 2
#     }

#     data['category'] = data['category'].map(category_mapping)
#     data['title_content'] = data['title'] + " " + data['content']
#     data.drop(['title','content'],axis = 1, inplace =True)
#     return data

In [9]:
#doc2vec 에 필요한 데이터 만들어줌

def make_doc2vec_data(data, column, t_document=False):
    data_doc = []
    for tag, doc in zip(data.index, data[column]):
        doc = doc.split(" ")
        data_doc.append(([tag], doc))
    if t_document:
        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
        return data
    else:
        return data_doc

In [10]:
#cosine similarity 구하기 , top 5 추천

def get_recommend_contents(user,data_doc,model):
    scores=[]

    for tags, text in data_doc:
        trained_doc_vec = model.dv[tags[0]]
        scores.append(cosine_similarity(user.reshape(-1,128),trained_doc_vec.reshape(-1,128)))

    scores = np.array(scores).reshape(-1)
    scores = np.argsort(-scores)[:5] #cosine similarity 가 높은 인덱스를 추출해서 가져온다.

    return data.loc[scores, :]

In [11]:
#content vector 기반의 user history 평균해서 user embedding 만들어 주는 함수

def make_user_embedding(index_list,data_doc,model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.dv[i])
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding,axis = 0)
    return user

In [12]:
# 사용자의 히스토리 내역 보여주는 함수

def view_user_history(data):
    print(data[['MainCategory','Content']])

In [13]:
data_doc_title_content_tag = make_doc2vec_data(data, 'Content',t_document=True)
data_doc_title_content = make_doc2vec_data(data, 'Content')
data_doc_title_content

## make Doc2Vec Models

In [31]:
#model 학습
make_doc2vec_models(data_doc_title_content_tag,tok = False)


In [34]:
user_category_1 = data.loc[random.sample(data.loc[data.MainCategory == '스포츠', :].index.values.tolist(),5),:]
user_category_2 = data.loc[random.sample(data.loc[data.MainCategory == '연예', :].index.values.tolist(),5),:]
user_category_3 = data.loc[random.sample(data.loc[data.MainCategory == '사회', :].index.values.tolist(),5),:]
user_category_4 = data.loc[random.sample(data.loc[data.MainCategory == '생활/문화', :].index.values.tolist(),5),:]
user_category_5 = data.loc[random.sample(data.loc[data.MainCategory == '경제', :].index.values.tolist(),5),:]
user_category_6 = data.loc[random.sample(data.loc[data.MainCategory == '정치', :].index.values.tolist(),5),:]
user_category_7 = data.loc[random.sample(data.loc[data.MainCategory == 'IT', :].index.values.tolist(),5),:]
user_category_8 = data.loc[random.sample(data.loc[data.MainCategory == '국제', :].index.values.tolist(),5),:]


In [44]:
view_user_history(user_category_8)

       MainCategory                                            Content
658967           국제  Iowa Gov. Kim Reynolds speaks to supporters du...
661282           국제  Republican Lee Murphy seen in this 2022 campai...
630167           국제  Englands Jack Grealish second left celebrates ...
624698           국제  Canadas Tajon Buchanan grimaces on the ground ...
615548           국제  Pope Francis meets with members of the Italian...


## load doc2vec models

In [45]:
model_title_content = Doc2Vec.load('./False_news_model.doc2vec')

In [57]:
# 사용자 히스토리를 가지고 User Embedding

user_1 = make_user_embedding(user_category_1.index.values.tolist(),data_doc_title_content,model_title_content)
user_2 = make_user_embedding(user_category_2.index.values.tolist(),data_doc_title_content,model_title_content)
user_3 = make_user_embedding(user_category_3.index.values.tolist(),data_doc_title_content,model_title_content)
user_4 = make_user_embedding(user_category_4.index.values.tolist(),data_doc_title_content,model_title_content)
user_5 = make_user_embedding(user_category_5.index.values.tolist(),data_doc_title_content,model_title_content)
user_6 = make_user_embedding(user_category_6.index.values.tolist(),data_doc_title_content,model_title_content)
user_7 = make_user_embedding(user_category_7.index.values.tolist(),data_doc_title_content,model_title_content)
user_8 = make_user_embedding(user_category_8.index.values.tolist(),data_doc_title_content,model_title_content)

In [58]:
user_1

array([-5.00858665e-01, -1.64297730e-01, -2.64968634e-01,  1.99933231e-01,
        2.40015075e-01, -2.62998603e-02,  1.51523173e-01,  2.36087735e-03,
       -2.84220695e-01,  3.14649522e-01, -3.50444131e-02, -2.50655234e-01,
       -9.60915685e-02,  1.67870909e-01, -2.29139045e-01, -1.11828241e-02,
        9.11272168e-02,  3.32633667e-02, -4.63434346e-02, -1.13441750e-01,
        1.62217170e-01, -1.32087529e-01,  2.40574926e-02,  3.08847010e-01,
       -2.17497587e-01,  1.92885295e-01,  2.50423223e-01,  2.00312182e-01,
        2.49833539e-01, -6.63963985e-03, -2.20583864e-02,  5.58530279e-02,
        4.54498202e-01,  6.60716668e-02,  1.40586391e-01,  3.91958237e-01,
        4.65672314e-01,  1.26839280e-01,  3.27177137e-01,  2.66723156e-01,
       -1.02511667e-01,  1.23063065e-01, -3.53927195e-01, -2.28673607e-01,
       -2.80501693e-01,  6.89029768e-02,  4.26459700e-01,  1.14274465e-01,
        3.52135330e-01, -1.62437826e-01,  1.37467727e-01,  2.34734371e-01,
        9.04941633e-02,  

In [61]:
# 추천
result = get_recommend_contents(user_1,data_doc_title_content,model_title_content)
pd.DataFrame(result.loc[:, ['MainCategory', 'Content']])

KeyboardInterrupt: 

## 뉴스 기사 제목과 본문 합치기 / mecab 형태소 분석